In [1]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates

from catboost import CatBoostRegressor
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
from datetime import timedelta, date

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [33]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")


## Dense

In [38]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']

col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        ('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)

In [39]:
cat = CatBoostRegressor(
    depth=12,
    iterations=1500,
    rsm=0.35,
    subsample=0.7
)

pipe = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
    ('model', cat)
])


pipe.fit(X, y)

predictions = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Learning rate set to 0.078527
0:	learn: 1.5680777	total: 33.9ms	remaining: 50.8s
1:	learn: 1.4848678	total: 63.1ms	remaining: 47.2s
2:	learn: 1.4095191	total: 91.4ms	remaining: 45.6s
3:	learn: 1.3419264	total: 118ms	remaining: 44.3s
4:	learn: 1.2803091	total: 149ms	remaining: 44.5s
5:	learn: 1.2258658	total: 180ms	remaining: 44.9s
6:	learn: 1.1776935	total: 210ms	remaining: 44.8s
7:	learn: 1.1313631	total: 241ms	remaining: 44.9s
8:	learn: 1.0908722	total: 271ms	remaining: 44.9s
9:	learn: 1.0546348	total: 299ms	remaining: 44.6s
10:	learn: 1.0186919	total: 328ms	remaining: 44.5s
11:	learn: 0.9889310	total: 359ms	remaining: 44.5s
12:	learn: 0.9598742	total: 385ms	remaining: 44.1s
13:	learn: 0.9357989	total: 420ms	remaining: 44.6s
14:	learn: 0.9137988	total: 450ms	remaining: 44.6s
15:	learn: 0.8890610	total: 477ms	remaining: 44.2s
16:	learn: 0.8674710	total: 506ms	remaining: 44.1s
17:	learn: 0.8493097	total: 534ms	remaining: 44s
18:	learn: 0.8304182	total: 566ms	remaining: 44.1s
19:	learn:

In [40]:
#catboost_predictions = pipe.predict(X)
catboost_predictions_train = pipe.predict(X_train)
catboost_predictions_test = pipe.predict(X_test)

In [41]:
pipe2 = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
])
X_train_prepro = pipe2.fit_transform(X_train)
X_test_prepro = pipe2.fit_transform(X_test)

In [42]:

X_train_with_predictions = np.hstack((X_train_prepro, catboost_predictions_train.reshape(-1, 1)))
X_test_with_predictions = np.hstack((X_test_prepro, catboost_predictions_test.reshape(-1, 1)))

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_with_predictions.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Pour la régression

model.compile(optimizer='adam', loss='mean_squared_error')

#early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(X_train_with_predictions, y_train, epochs=100, batch_size=64,validation_data=(X_test_with_predictions, y_test))#callbacks=[early_stopping], )

mse = model.evaluate(X_test_with_predictions, y_test)
# Évaluation du modèle
model.evaluate(X_test_with_predictions, y_test)

Epoch 1/100
7133/7133 [==============================] - 4s 613us/step - loss: 15.5800 - val_loss: 0.3882
Epoch 2/100
7133/7133 [==============================] - 4s 613us/step - loss: 0.4893 - val_loss: 0.1201
Epoch 3/100
7133/7133 [==============================] - 4s 613us/step - loss: 0.1363 - val_loss: 0.1616
Epoch 4/100
7133/7133 [==============================] - 4s 610us/step - loss: 0.1141 - val_loss: 0.1227
Epoch 5/100
7133/7133 [==============================] - 4s 609us/step - loss: 0.2641 - val_loss: 0.0736
Epoch 6/100
7133/7133 [==============================] - 4s 624us/step - loss: 0.1029 - val_loss: 0.0932
Epoch 7/100
7133/7133 [==============================] - 4s 631us/step - loss: 0.1081 - val_loss: 0.1585
Epoch 8/100
7133/7133 [==============================] - 5s 655us/step - loss: 0.0951 - val_loss: 0.0720
Epoch 9/100
7133/7133 [==============================] - 4s 630us/step - loss: 0.1005 - val_loss: 0.0925
Epoch 10/100
7133/7133 [==============================

0.07579505443572998

In [43]:
y_pred = model.predict(X_test_with_predictions)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

1260/1260 [==============================] - 0s 315us/step
Root Mean Squared Error: 0.27530904722803934


In [44]:
X_final_test_scaled = pipe.predict(X_final_test)
X_final_test_with_predictions = np.hstack((pipe2.fit_transform(X_final_test), X_final_test_scaled.reshape(-1, 1)))
submission = model.predict(X_final_test_with_predictions)
pd.DataFrame(submission, columns=["log_bike_count"]).rename_axis('Id').to_csv('submission56_deep_101223.csv')

1608/1608 [==============================] - 0s 298us/step


In [13]:
best_sub = pd.read_csv('submission17_91223.csv')
rmse = np.sqrt(mean_squared_error(best_sub['log_bike_count'], predictions))
print(f'RMSE:{rmse}')

RMSE:0.15017399128824682


In [14]:
pd.DataFrame(predictions, columns=["log_bike_count"]).rename_axis('Id').to_csv('submission34_deep_101223.csv')